In [3]:
using CSV, DataFrames, Statistics, Dates, Gadfly, GLM, Distributions, LinearAlgebra, DecisionTree, ScikitLearn
include("Utils.jl")

categorizeMax (generic function with 1 method)

In [4]:
data_surverse = CSV.read("data/surverses.csv",missingstring="-99999")
first(data_surverse,5)

,NO_OUVRAGE,DATE,SURVERSE,RAISON
,String,Date,Int64⍰,String⍰
1,0642-01D,2013-05-01,0,missing
2,0642-01D,2013-05-02,0,missing
3,0642-01D,2013-05-03,0,missing
4,0642-01D,2013-05-04,0,missing
5,0642-01D,2013-05-05,0,missing


In [5]:
data_surverse = filter(row -> month(row.DATE) > 4, data_surverse) 
data_surverse = filter(row -> month(row.DATE) < 11, data_surverse) 
first(data_surverse,5)

,NO_OUVRAGE,DATE,SURVERSE,RAISON
,String,Date,Int64⍰,String⍰
1,0642-01D,2013-05-01,0,missing
2,0642-01D,2013-05-02,0,missing
3,0642-01D,2013-05-03,0,missing
4,0642-01D,2013-05-04,0,missing
5,0642-01D,2013-05-05,0,missing


In [6]:
raison = coalesce.(data_surverse[:,:RAISON],"Inconnue")
data_surverse[!,:RAISON] = raison
first(data_surverse,5)

,NO_OUVRAGE,DATE,SURVERSE,RAISON
,String,Date,Int64⍰,String
1,0642-01D,2013-05-01,0,Inconnue
2,0642-01D,2013-05-02,0,Inconnue
3,0642-01D,2013-05-03,0,Inconnue
4,0642-01D,2013-05-04,0,Inconnue
5,0642-01D,2013-05-05,0,Inconnue


In [7]:
data_surverse = filter(row -> row.RAISON ∈ ["P","Inconnue","TS"], data_surverse) 
select!(data_surverse, [:NO_OUVRAGE, :DATE, :SURVERSE])
first(data_surverse,5)

,NO_OUVRAGE,DATE,SURVERSE
,String,Date,Int64⍰
1,0642-01D,2013-05-01,0
2,0642-01D,2013-05-02,0
3,0642-01D,2013-05-03,0
4,0642-01D,2013-05-04,0
5,0642-01D,2013-05-05,0


In [8]:
surverse_df = dropmissing(data_surverse, disallowmissing=true)
first(surverse_df,5)

,NO_OUVRAGE,DATE,SURVERSE
,String,Date,Int64
1,0642-01D,2013-05-01,0
2,0642-01D,2013-05-02,0
3,0642-01D,2013-05-03,0
4,0642-01D,2013-05-04,0
5,0642-01D,2013-05-05,0


In [9]:
data_precipitations = CSV.read("data/precipitations.csv",missingstring="-99999")
rename!(data_precipitations, Symbol("St-Hubert")=>:StHubert)
first(data_precipitations,5)

,date,heure,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,2013-01-01,0,0,0,0,0,missing
2,2013-01-01,1,0,0,0,0,missing
3,2013-01-01,2,0,0,0,0,missing
4,2013-01-01,3,0,0,0,0,missing
5,2013-01-01,4,0,0,0,0,missing


In [10]:
data_precipitations = filter(row -> month(row.date) > 4, data_precipitations) 
data_precipitations = filter(row -> month(row.date) < 11, data_precipitations) 
rowMean(data_precipitations, 5, 2)
first(data_precipitations,5)

,date,heure,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,2013-05-01,0,0,0,0,0,0
2,2013-05-01,1,0,0,0,0,0
3,2013-05-01,2,0,0,0,0,0
4,2013-05-01,3,0,0,0,0,0
5,2013-05-01,4,0,0,0,0,0


In [11]:
data_ouvrage = CSV.read("data/ouvrages-surverses.csv",missingstring="")

# Extract speficic ouvrage
df_ouvrage = filter(row ->
    row.ID_ouvrage == "3260-01D" ||
    row.ID_ouvrage == "3350-07D" ||
    row.ID_ouvrage == "4240-01D" ||
    row.ID_ouvrage == "4350-01D" ||
    row.ID_ouvrage == "4380-01D", data_ouvrage)

df_ouvrage = df_ouvrage[:,[:ID_ouvrage, Symbol("Trop-Plein X"), Symbol("Trop-Plein Y"), Symbol("Trop-Plein Z")]]
rename!(df_ouvrage, Symbol("ID_ouvrage")=>:NO_OUVRAGE)

meanHeight = mean(skipmissing(df_ouvrage[Symbol("Trop-Plein Z")]))

# Replace missing height with it's mean
df_ouvrage[ismissing.(df_ouvrage[Symbol("Trop-Plein Z")]), Symbol("Trop-Plein Z")] .= meanHeight

df_ouvrage

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[11]:13
└ @ Core In[11]:13
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[11]:15
└ @ Core In[11]:15


,NO_OUVRAGE,Trop-Plein X,Trop-Plein Y,Trop-Plein Z
,String,Float64,Float64,Float64⍰
1,3260-01D,2.98543e5,5.05676e6,20.17
2,3350-07D,2.89801e5,5.04516e6,20.75
3,4240-01D,3.05755e5,5.05665e6,11.91
4,4350-01D,3.00503e5,5.03991e6,17.61
5,4380-01D,2.9982e5,5.03643e6,17.61


In [14]:
# Analyse exploratoire

# Faire la somme des precipitations par rapport a la date
pcp_sum = by(data_precipitations, :date,  McTavish = :McTavish=>sum, Bellevue = :Bellevue=>sum, 
           Assomption = :Assomption=>sum, Trudeau = :Trudeau=>sum, StHubert = :StHubert=>sum)

pcp_max = by(data_precipitations, :date,  McTavish = :McTavish=>maximum, Bellevue = :Bellevue=>maximum, 
   Assomption = :Assomption=>maximum, Trudeau = :Trudeau=>maximum, StHubert = :StHubert=>maximum)

df_surve = filter(row ->
    row.NO_OUVRAGE == "3260-01D" ||
    row.NO_OUVRAGE == "3350-07D" ||
    row.NO_OUVRAGE == "4240-01D" ||
    row.NO_OUVRAGE == "4350-01D" ||
    row.NO_OUVRAGE == "4380-01D", surverse_df)

rename!(df_surve, Symbol("DATE")=>:date)
rename!(pcp_max,
    Symbol("McTavish")=>:McTavishMax,
    Symbol("Bellevue")=>:BellevueMax,
    Symbol("Assomption")=>:AssomptionMax,
    Symbol("Trudeau")=>:TrudeauMax,
    Symbol("StHubert")=>:StHubertMax)

enhanced_surve = join(df_surve, pcp_sum, on=:date, kind=:inner)
enhanced_surve = join(enhanced_surve, pcp_max, on=:date, kind=:inner)
enhanced_surve = join(enhanced_surve, df_ouvrage, on=:NO_OUVRAGE, kind=:left)

# enhanced_train_data, enhanced_test_data = partitionTrainTest(enhanced_surve, 0.7)

train_data = convert(Matrix, enhanced_surve[:,
           [:McTavish,
            :Bellevue,
            :Assomption,
            :Trudeau,
            :StHubert,
            :McTavishMax,          
            :BellevueMax,          
            :AssomptionMax,        
            :TrudeauMax,           
            :StHubertMax,   
            Symbol("Trop-Plein X"),
            Symbol("Trop-Plein Y"),
            Symbol("Trop-Plein Z")]])
names(enhanced_surve)
train_result = convert(Array{Float64}, enhanced_surve[:,:SURVERSE])

model = build_forest(train_result, train_data)

Ensemble of Decision Trees
Trees:      10
Avg Leaves: 93.6
Avg Depth:  12.2

In [13]:
# Test accuracy (Il faut trouver une meilleur maniere pour faire le cross validation)
# Cette maniere n'a pas l'air legit
# test_data = convert(Matrix, enhanced_test_data[:,[:McTavish, :Bellevue, :Assomption, :Trudeau, :StHubert]])
# test_result = convert(Array{Float64}, enhanced_test_data[:,:SURVERSE])
# accuracy = nfoldCV_forest(test_result, test_data, 10, 2)

In [15]:
### Retrieve test data
test = CSV.read("data/test.csv");

test_df_surve = filter(row ->
    row.NO_OUVRAGE == "3260-01D" ||
    row.NO_OUVRAGE == "3350-07D" ||
    row.NO_OUVRAGE == "4240-01D" ||
    row.NO_OUVRAGE == "4350-01D" ||
    row.NO_OUVRAGE == "4380-01D", test)

rename!(test_df_surve, Symbol("DATE")=>:date) 

test_df_serve = join(test_df_surve, pcp_sum, on=:date, kind=:inner)
test_df_serve = join(test_df_serve, pcp_max, on=:date, kind=:inner)
test_df_serve = join(test_df_serve, df_ouvrage, on=:NO_OUVRAGE, kind=:left)

test_data = convert(Matrix, test_df_serve[:,
        [:McTavish,
            :Bellevue,
            :Assomption,
            :Trudeau,
            :StHubert,
            :McTavishMax,          
            :BellevueMax,          
            :AssomptionMax,        
            :TrudeauMax,           
            :StHubertMax,   
            Symbol("Trop-Plein X"),
            Symbol("Trop-Plein Y"),
            Symbol("Trop-Plein Z")]])

n = size(test_df_serve, 1)
result = Array{Float64}(undef, n)

for i = 1:n
    result[i] = apply_forest(model, test_data[i, :])
end

Y = zeros(Int64, n)
Y[result.>.5] .= 1
Y

283-element Array{Int64,1}:
 0
 0
 1
 0
 0
 1
 0
 0
 0
 1
 0
 0
 1
 ⋮
 0
 1
 1
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [21]:
# RSME - SKlearn

rsme_data_test = convert(Matrix, enhanced_surve[:,
        [:McTavish,
        :Bellevue,
        :Assomption,
        :Trudeau,
        :StHubert,
        :McTavishMax,          
        :BellevueMax,          
        :AssomptionMax,        
        :TrudeauMax,           
        :StHubertMax,   
        Symbol("Trop-Plein X"),
        Symbol("Trop-Plein Y"),
        Symbol("Trop-Plein Z")]])

rsme_data_result = convert(Array{Float64}, enhanced_surve[:,:SURVERSE])

n = size(enhanced_surve, 1)
result = Array{Float64}(undef, n)

for i = 1:n
    result[i] = apply_forest(model, rsme_data_test[i, :])
end

Y = zeros(Int64, n)
Y[result.>.5] .= 1
Y
rsme = mean_squared_error(rsme_data_result, Y)

0.02412280701754386

In [136]:
ID = Array{String}(undef, n)
for i = 1:n
    ID[i] = string(test[i,:NO_OUVRAGE], "_", Dates.format(test[i,:DATE], "yyyy-mm-dd"))
end
YTotalBool = Array{String}(undef, n)

for i = 1:n
    if Y[i] == 0
        YTotalBool[i] = "FALSE"
    else
        YTotalBool[i] = "TRUE"
    end
end

prediction = DataFrame(ID = ID, Surverse = YTotalBool)
CSV.write("sampleSubmissionTest.csv",prediction)

"sampleSubmissionTest.csv"